In [1]:
import os
import sys
import json

from hdlConvertor import HdlConvertor
from hdlConvertorAst.language import Language
from hdlConvertorAst.to.vhdl import ToVhdl2008
from hdlConvertorAst.hdlAst import HdlValueInt, HdlValueId

In [2]:
h = HdlConvertor()
h.parse?

Signature:
h.parse(
    filenames,
    language,
    incdirs,
    encoding='utf-8',
    hierarchyOnly=False,
    debug=True,
)
Docstring:
:param filenames: sequence of filenames or filename
:type filename: Union[str, List[str]]
:param language: hdlConvertor.language.Language enum value
:param incdirs: list of include directories
:param encoding: character encoding of input data
:param hierarchyOnly: if True only names of components and modules are parsed
:param debug: if True the debug logging is enabled
:return: HdlContext instance
Type:      method

In [56]:
class VhdlGenericModifier:
    def __init__(self, vhdl_file, include_dirs=[], language=Language.VHDL_2008):
        """Initialize with the VHDL file to be modified"""
        self.vhdl_file = vhdl_file
        self.converter = HdlConvertor()
        
        # Parse the VHDL file using proper arguments
        self.ast = self.converter.parse(
            filenames=[vhdl_file],
            language=language,
            incdirs=include_dirs,
            hierarchyOnly=False
        )
        self.entity_name = os.path.splitext(os.path.basename(vhdl_file))[0]
        
    def get_generics(self):
        """Extract all generics and their default values"""
        generics = {}
        
        for obj in self.ast.objs:
            # Find entity declarations
            if hasattr(obj, 'dec') and obj.dec is not None:
                module_dec = obj.dec
                # Check for generic parameters
                if hasattr(module_dec, 'params') and module_dec.params:
                    for param in module_dec.params:
                        value = None
                        if hasattr(param, 'value') and param.value is not None:
                            if hasattr(param.value, 'val'):
                                value = param.value.val
                            else:
                                value = str(param.value)
                                
                        type_name = None
                        if hasattr(param, 'type') and param.type is not None:
                            if hasattr(param.type, 'name'):
                                type_name = param.type.name
                            else:
                                type_name = str(param.type)
                                
                        generics[param.name] = {
                            'value': value,
                            'type': type_name
                        }
        return generics
    
    def modify_generic(self, generic_name, new_value):
        """Modify a specific generic parameter"""
        modified = False
        
        for obj in self.ast.objs:
            if hasattr(obj, 'dec') and obj.dec is not None:
                module_dec = obj.dec
                if hasattr(module_dec, 'params') and module_dec.params:
                    for param in module_dec.params:
                        if param.name == generic_name:
                            old_value = None
                            if hasattr(param, 'value') and param.value is not None:
                                if hasattr(param.value, 'val'):
                                    old_value = param.value.val
                                else:
                                    old_value = str(param.value)
                            
                            # Create appropriate value node based on type
                            if isinstance(new_value, int):
                                param.value = HdlValueInt(new_value, None)
                            else:
                                param.value = HdlValueId(str(new_value))
                                
                            modified = True
                            print(f"Modified {generic_name} from {old_value} to {new_value}")
        
        return modified
    
    def generate_vhdl(self, output_file=None):
        """Generate VHDL code from the modified AST"""
        vhdl_serializer = ToVhdl2008()
        modified_vhdl = vhdl_serializer.visit_HdlContext(self.ast)
        
        if output_file is None:
            basename = os.path.splitext(self.vhdl_file)[0]
            output_file = f"{basename}_modified.vhd"
            
        with open(output_file, 'w') as f:
            f.write(modified_vhdl)
            
        return output_file
    
    def export_for_scilab(self, output_dir="."):
        """Export metadata for Scilab integration"""
        generics = self.get_generics()
        
        metadata = {
            "entity": self.entity_name,
            "source_file": self.vhdl_file,
            "generics": generics,
            "hdl_type": "vhdl"
        }
        
        os.makedirs(output_dir, exist_ok=True)
        json_file = os.path.join(output_dir, f"{self.entity_name}_metadata.json")
        
        with open(json_file, 'w') as f:
            json.dump(metadata, f, indent=2)
            
        return json_file


In [57]:
# Example that follows the proper hdlConvertor usage pattern
def modify_vhdl_entity(input_file, output_file=None, generics=None):
    """
    Modify VHDL entity generics and generate new file
    
    Args:
        input_file: Path to input VHDL file
        output_file: Path to output file (optional)
        generics: Dictionary of generic name to new value
    """
    modifier = VhdlGenericModifier(input_file)
    
    # Print original generics
    print("Original generics:")
    for name, details in modifier.get_generics().items():
        print(f"  {name} = {details['value']} ({details['type']})")
    
    # Apply modifications
    if generics:
        for name, value in generics.items():
            modifier.modify_generic(name, value)
    
    # Generate modified VHDL
    result_file = modifier.generate_vhdl(output_file)
    print(f"Generated modified VHDL: {result_file}")
    
    return result_file

# Example call
# modify_vhdl_entity("adder.vhd", generics={"WIDTH": 32, "RESET_ACTIVE": "'0'"})


In [73]:
from hdlConvertorAst.to.hdl_ast_visitor import HdlAstVisitor
from hdlConvertorAst.hdlAst import HdlValueId, HdlOp

class PortNamesToLowerCase(HdlAstVisitor):
    """
    Make port names lower case HDL AST

    :note: this is just a stupid rewrite of all variable names to lower case
    """
    def visit_HdlIdDef(self, o):
        o.name = o.name.lower()

    def visit_HdlStmProcess(self, o):
        if o.sensitivity:
            o.sensitivity = [self.visit_iHdlExpr(s) for s in o.sensitivity]
        self.visit_iHdlStatement(o.body)

    def visit_iHdlExpr(self, o):
        if isinstance(o, HdlValueId):
            return HdlValueId(o.val.lower(), obj=o.obj)
        elif isinstance(o, HdlOp):
            o.ops = [self.visit_iHdlExpr(s) for s in o.ops]
            return o
        else:
            return super(PortNamesToLowerCase, self).visit_iHdlExpr(o)

    def visit_HdlStmIf(self, o):
        o.cond = self.visit_iHdlExpr(o.cond)
        super(PortNamesToLowerCase, self).visit_HdlStmIf(o)

    def visit_HdlStmAssign(self, o):
        o.src = self.visit_iHdlExpr(o.src)
        o.dst = self.visit_iHdlExpr(o.dst)


In [74]:
TEST_DIR = os.path.join("..", "tests", "vhdl")

filenames = [os.path.join(TEST_DIR, "generate_for_and_if.vhd"), ]
include_dirs = []

with open(modifer.vhdl_file, 'r') as f:
    ...
    # print("".join(f.readlines()))
    

tv = ToVhdl2008(sys.stdout)
d = HdlConvertor().parse(
            filenames=filenames[0],
            language=Language.VHDL_2008,
            incdirs=include_dirs,
            hierarchyOnly=False
        )
PortNamesToLowerCase().visit_HdlContext(d)
tv.visit_HdlContext(d)

LIBRARY IEEE;
USE IEEE.STD_LOGIC_1164.ALL;
USE IEEE.NUMERIC_STD.ALL;
USE IEEE.MATH_REAL.ALL;
LIBRARY BigRISC_LIB;
USE BigRISC_LIB.BigRISC_CONFIG.ALL;
USE BigRISC_LIB.BigRISC_HELPERS.ALL;
ENTITY Shifter IS
    GENERIC(
        size : NATURAL := WORD_SIZE;
        shift_size : NATURAL := LOG2(WORD_SIZE);
        pipelined : NATURAL := 2
    );
    PORT(
        clk_in : IN STD_LOGIC;
        reset_in : IN STD_LOGIC;
        enable_in : IN STD_LOGIC;
        operand_in : IN STD_LOGIC_VECTOR(SIZE - 1 DOWNTO 0);
        shift_in : IN STD_LOGIC_VECTOR(SHIFT_SIZE - 1 DOWNTO 0);
        shifttype_in : IN STD_LOGIC_VECTOR(SHIFT_SELECT_SIZE - 1 DOWNTO 0);
        result_out : OUT STD_LOGIC_VECTOR(SIZE - 1 DOWNTO 0)
    );
END ENTITY;

ARCHITECTURE arch OF Shifter IS
    CONSTANT stage_stages : INTEGER := integer(ceil(real(SHIFT_SIZE) / real(PIPELINED)));
    TYPE stagearray IS ARRAY (NATURAL RANGE <>) OF STD_LOGIC_VECTOR(SIZE - 1 DOWNTO 0);
    SIGNAL stages : StageArray(SHIFT_SIZE DOWNTO 0);
  

[{'__class__': 'HdlLibrary',
  'name': {'__class__': 'str', 'val': 'IEEE'},
  'position': (1, 9, 1, 12)},
 {'__class__': 'HdlImport',
  'path': {'__class__': 'list', 'items': [...]},
  'position': (2, 5, 2, 27)},
 {'__class__': 'HdlImport',
  'path': {'__class__': 'list', 'items': [...]},
  'position': (3, 5, 3, 24)},
 {'__class__': 'HdlImport',
  'path': {'__class__': 'list', 'items': [...]},
  'position': (4, 5, 4, 22)},
 {'__class__': 'HdlLibrary',
  'name': {'__class__': 'str', 'val': 'BigRISC_LIB'},
  'position': (6, 9, 6, 19)},
 {'__class__': 'HdlImport',
  'path': {'__class__': 'list', 'items': [...]},
  'position': (7, 5, 7, 34)},
 {'__class__': 'HdlImport',
  'path': {'__class__': 'list', 'items': [...]},
  'position': (8, 5, 8, 35)},
 {'__class__': 'HdlModuleDec',
  'declaration_only': False,
  'name': {'__class__': 'str', 'val': 'Shifter'},
  'objs': [],
  'params': [{...}, {...}, {...}],
  'ports': [{...}, {...}, {...}, {...}, {...}, {...}, {...}],
  'position': (10, 1, 33,

In [61]:
modifer.ast.objs

[{'__class__': 'HdlLibrary',
  'name': {'__class__': 'str', 'val': 'IEEE'},
  'position': (1, 9, 1, 12)},
 {'__class__': 'HdlImport',
  'path': {'__class__': 'list',
           'items': ['IEEE', 'STD_LOGIC_1164', {'__class__': 'HdlAll'}]},
  'position': (2, 5, 2, 27)},
 {'__class__': 'HdlImport',
  'path': {'__class__': 'list',
           'items': ['IEEE', 'NUMERIC_STD', {'__class__': 'HdlAll'}]},
  'position': (3, 5, 3, 24)},
 {'__class__': 'HdlImport',
  'path': {'__class__': 'list',
           'items': ['IEEE', 'MATH_REAL', {'__class__': 'HdlAll'}]},
  'position': (4, 5, 4, 22)},
 {'__class__': 'HdlLibrary',
  'name': {'__class__': 'str', 'val': 'BigRISC_LIB'},
  'position': (6, 9, 6, 19)},
 {'__class__': 'HdlImport',
  'path': {'__class__': 'list',
           'items': ['BigRISC_LIB', 'BigRISC_CONFIG', {'__class__': 'HdlAll'}]},
  'position': (7, 5, 7, 34)},
 {'__class__': 'HdlImport',
  'path': {'__class__': 'list',
           'items': ['BigRISC_LIB', 'BigRISC_HELPERS', {'__class__

In [64]:
tv = ToVhdl2008(sys.stdout)
tv.visit_HdlContext(modifer.ast)


LIBRARY IEEE;
USE IEEE.STD_LOGIC_1164.ALL;
USE IEEE.NUMERIC_STD.ALL;
USE IEEE.MATH_REAL.ALL;
LIBRARY BigRISC_LIB;
USE BigRISC_LIB.BigRISC_CONFIG.ALL;
USE BigRISC_LIB.BigRISC_HELPERS.ALL;
ENTITY Shifter IS
    GENERIC(
        SIZE : NATURAL := WORD_SIZE;
        SHIFT_SIZE : NATURAL := LOG2(WORD_SIZE);
        PIPELINED : NATURAL := 2
    );
    PORT(
        Clk_in : IN STD_LOGIC;
        Reset_in : IN STD_LOGIC;
        Enable_in : IN STD_LOGIC;
        Operand_in : IN STD_LOGIC_VECTOR(SIZE - 1 DOWNTO 0);
        Shift_in : IN STD_LOGIC_VECTOR(SHIFT_SIZE - 1 DOWNTO 0);
        ShiftType_in : IN STD_LOGIC_VECTOR(SHIFT_SELECT_SIZE - 1 DOWNTO 0);
        Result_out : OUT STD_LOGIC_VECTOR(SIZE - 1 DOWNTO 0)
    );
END ENTITY;

ARCHITECTURE arch OF Shifter IS
    CONSTANT STAGE_STAGES : INTEGER := integer(ceil(real(SHIFT_SIZE) / real(PIPELINED)));
    TYPE StageArray IS ARRAY (NATURAL RANGE <>) OF STD_LOGIC_VECTOR(SIZE - 1 DOWNTO 0);
    SIGNAL Stages : StageArray(SHIFT_SIZE DOWNTO 0);
  

[{'__class__': 'HdlLibrary',
  'name': {'__class__': 'str', 'val': 'IEEE'},
  'position': (1, 9, 1, 12)},
 {'__class__': 'HdlImport',
  'path': {'__class__': 'list', 'items': [...]},
  'position': (2, 5, 2, 27)},
 {'__class__': 'HdlImport',
  'path': {'__class__': 'list', 'items': [...]},
  'position': (3, 5, 3, 24)},
 {'__class__': 'HdlImport',
  'path': {'__class__': 'list', 'items': [...]},
  'position': (4, 5, 4, 22)},
 {'__class__': 'HdlLibrary',
  'name': {'__class__': 'str', 'val': 'BigRISC_LIB'},
  'position': (6, 9, 6, 19)},
 {'__class__': 'HdlImport',
  'path': {'__class__': 'list', 'items': [...]},
  'position': (7, 5, 7, 34)},
 {'__class__': 'HdlImport',
  'path': {'__class__': 'list', 'items': [...]},
  'position': (8, 5, 8, 35)},
 {'__class__': 'HdlModuleDec',
  'declaration_only': False,
  'name': {'__class__': 'str', 'val': 'Shifter'},
  'objs': [],
  'params': [{...}, {...}, {...}],
  'ports': [{...}, {...}, {...}, {...}, {...}, {...}, {...}],
  'position': (10, 1, 33,